### Playfair密码（单表代替密码，多字母代替密码）

**1. 根据密钥词生成 5 x 5 矩阵**

**1. 将明文分割成字母对，若字母对字母相同，在中间加一个填充字母，如x。`eg:balloon先变成ba,lx,lo,on`**

**2. 对每个字母对——**
   * 若字母在矩阵的同一行，则每个字母由其右边的字母代替，`eg: AR->RM`
   * 若字母在矩阵的同一列，则每个字母由其下边的字母代替，`eg: MU->CM`
   * 否则对每个字母，其行为密文的所在行，另一字母的列为密文所在列，`eg:HS->BP`

In [7]:
import numpy as np
import re 
#使用字典存储
alphabet = 'abcdefghiklmnopqrstuvwxyz'
def create_matrix(word):
    res = {} #key为字母/下标，value为下标/字母
    count = 0
    for each in word.lower() + alphabet:
        if each == 'j': each = 'i'
        if res.get(each,-1) == -1: 
            res[each] = count
            res[count] = each
            count += 1
    return res

def crypt_one_word(word, mat):
    word = re.sub(r'j', 'i', word.lower())
    cut, res = word[0], ''
    for i in range(1, len(word)):
        cut += word[i] if len(cut) % 2 == 0 or word[i] != cut[-1] else 'x' + word[i]
    if len(cut) % 2: cut += 'x'
    for i in range(0,len(cut),2):
        pos1, pos2 = mat[cut[i]], mat[cut[i+1]]
        if pos1 // 5 == pos2 // 5:
            res += mat[pos1 // 5 * 5 + (pos1 + 1) % 5] + mat[pos2 // 5 * 5 + (pos2 + 1) % 5]
        elif pos1 % 5 == pos2 % 5:
            res += mat[(pos1 // 5 + 1) % 5 * 5 + pos1 % 5] + mat[(pos2 // 5 + 1) % 5 * 5 + pos1 % 5]
        else:
            res += mat[pos1 // 5 * 5 + pos2 % 5] + mat[pos2 // 5 * 5 + pos1 % 5]
    return res
            
def crypt_sentence(sentence, mat):
    res = sentence
    for word in re.findall(r'[a-zA-Z]+', sentence):
        res = re.sub('\\b' + word + '\\b', crypt_one_word(word, mat), res)
    return res

def decrypt_one_word(word, mat):
    res = ''
    for i in range(0,len(word),2):
        pos1, pos2 = mat[word[i]], mat[word[i+1]]
        if pos1 // 5 == pos2 // 5:
            res += mat[pos1 // 5 * 5 + (pos1 - 1) % 5] + mat[pos2 // 5 * 5 + (pos2 - 1) % 5]
        elif pos1 % 5 == pos2 % 5:
            res += mat[(pos1 // 5 - 1) % 5 * 5 + pos1 % 5] + mat[(pos2 // 5 - 1) % 5 * 5 + pos1 % 5]
        else:
            res += mat[pos1 // 5 * 5 + pos2 % 5] + mat[pos2 // 5 * 5 + pos1 % 5]
    return res

def decrypt_sentence(sentence, mat):
    res = sentence
    for word in re.findall(r'[a-zA-Z]+', sentence):
        res = re.sub('\\b' + word + '\\b', decrypt_one_word(word, mat), res)
    return res

In [8]:
matrix = create_matrix('monarchy')

In [11]:
sentence = 'i like this ball!'
C = crypt_sentence(sentence, matrix)
P = decrypt_sentence(C, matrix)
print("明文：" + sentence)
print("密文：" + C)
print("解密：" + P)

明文：i like this ball!
密文：sa seef pdsx ibsusu!
解密：ix like this balxlx!
